In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
import os
from collections import Counter
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import metrics
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge


# Epistasis plots

In [83]:
directory = '../data/ProteinGym_subs_apr_2024'
suitable_datasets = []
for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    if os.path.isfile(file_path):
        df = pd.read_csv(file_path)
        if 'mutant' in df.columns:
            mutations = df['mutant']
            for mut in mutations:
                if len(mut.split(':')) > 1:
                    suitable_datasets.append(file_path)
                    break
        else:
            print(filename, df.columns)

In [84]:
# Specify the file path
file_path = '../selected_datasets.txt'

# Open the file in write mode and write the strings line by line
with open(file_path, 'w') as file:
    for string in suitable_datasets:
        file.write(string.split('/')[-1].split('.')[0] + '\n')

In [7]:
for dataset in suitable_datasets:
    name = dataset.split('/')[-1].split('.')[0]
    df = pd.read_csv(dataset)
    indices = {}
    for ind, row in df.iterrows():
        mutations = row['mutant'].split(':')
        if len(mutations) == 2:
            indices[(mutations[0], mutations[1], ind)] = [-1, -1]

    for ind, row in df.iterrows():
        mutations = row['mutant'].split(':')
        if len(mutations) == 1:
            for key, value in indices.items():
                if key[0] == mutations[0]:
                    indices[key][0] = ind
                if key[1] == mutations[0]:
                    indices[key][1] = ind

    dic_final = {}
    for key, value in indices.items():
        if value[0] != -1 and value[1] != -1:
            dic_final[key] = value

    #df['DMS_score_log'] = np.log1p(df['DMS_score'])
    scaler = MinMaxScaler()
    df['DMS_score_scaled'] = scaler.fit_transform(df['DMS_score'].values.reshape(-1, 1))

    x = []
    y = []
    selected = []
    for key, value in dic_final.items():
        ind = key[2]
        x.append(df.loc[ind, 'DMS_score_scaled'])
        selected.append(key[0] + ':' + key[1])
        ind_first = value[0]
        ind_second = value[1]
        y.append(df.loc[ind_first, 'DMS_score_scaled'] * df.loc[ind_second, 'DMS_score_scaled'])

    # Calculate distances from the line y = x
    distances = np.abs(np.array(y) - np.array(x)) / np.sqrt(2)
    # Calculate mean and standard deviation of the distances
    mean_distance = np.mean(distances)
    std_distance = np.std(distances)
    # Define a threshold for "distinct" points (e.g., 2 standard deviations from the mean)
    threshold = mean_distance + 2 * std_distance
    # Identify points that are "distinct"
    distinct_points = distances > threshold
    distinct_x = np.array(x)[distinct_points.tolist()]
    distinct_y = np.array(y)[distinct_points.tolist()]
    selected_mutations = [element for element, mask in zip(selected, distinct_points.tolist()) if mask]

    cmap = plt.get_cmap('Paired')
    plt.figure(figsize=[8,8])
    plt.scatter(x, y, color=cmap(1), alpha=0.8, label='near points')
    plt.scatter(distinct_x, distinct_y, color=cmap(0), alpha=0.8, label='distinct points')
    plt.plot([0, 1], [0, 1], color='grey', alpha=0.8, linestyle='--', label='y=x')
    plt.xlabel('Double mutation', fontsize=14)
    plt.ylabel('Multiplication of single mutations', fontsize=14)
    plt.title(f'Measured fluorescence values {name}', fontsize=16)
    plt.legend(prop={'size': 14})
    plt.grid(True)
    plt.savefig(f'../figures/plots_datasets/2std/{name}.svg')
    plt.close()

In [ ]:
plt.rcParams.update({'font.size': 14})
subsets = []
start = 0
while start != 60:
    subsets.append(suitable_datasets[start:start+12])
    start += 12
subsets.append(suitable_datasets[start:])
for num in range(len(subsets)):
    subset = subsets[num]
    fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(20, 30))
    for index in range(len(subset)):
        dataset = subset[index]
        name = dataset.split('/')[-1].split('.')[0]
        df = pd.read_csv(dataset)
        indices = {}
        for ind, row in df.iterrows():
            mutations = row['mutant'].split(':')
            if len(mutations) == 2:
                indices[(mutations[0], mutations[1], ind)] = [-1, -1]

        for ind, row in df.iterrows():
            mutations = row['mutant'].split(':')
            if len(mutations) == 1:
                for key, value in indices.items():
                    if key[0] == mutations[0]:
                        indices[key][0] = ind
                    if key[1] == mutations[0]:
                        indices[key][1] = ind

        dic_final = {}
        for key, value in indices.items():
            if value[0] != -1 and value[1] != -1:
                dic_final[key] = value

        #df['DMS_score_log'] = np.log1p(df['DMS_score'])
        scaler = MinMaxScaler()
        df['DMS_score_scaled'] = scaler.fit_transform(df['DMS_score'].values.reshape(-1, 1))

        x = []
        y = []
        selected = []
        for key, value in dic_final.items():
            ind = key[2]
            x.append(df.loc[ind, 'DMS_score_scaled'])
            selected.append(key[0] + ':' + key[1])
            ind_first = value[0]
            ind_second = value[1]
            y.append(df.loc[ind_first, 'DMS_score_scaled'] * df.loc[ind_second, 'DMS_score_scaled'])

        # Calculate distances from the line y = x
        distances = np.abs(np.array(y) - np.array(x)) / np.sqrt(2)
        # Calculate mean and standard deviation of the distances
        mean_distance = np.mean(distances)
        std_distance = np.std(distances)
        # Define a threshold for "distinct" points (e.g., 2 standard deviations from the mean)
        threshold = mean_distance + 1 * std_distance
        # Identify points that are "distinct"
        distinct_points = distances > threshold
        distinct_x = np.array(x)[distinct_points.tolist()]
        distinct_y = np.array(y)[distinct_points.tolist()]
        selected_mutations = [element for element, mask in zip(selected, distinct_points.tolist()) if mask]

        i = index // 3
        j = index % 3
        cmap = plt.get_cmap('Paired')
        axes[i, j].scatter(x, y, color=cmap(1), alpha=0.8, label='near points')
        axes[i, j].scatter(distinct_x, distinct_y, color=cmap(0), alpha=0.8, label='distinct points')
        axes[i, j].plot([0, 1], [0, 1], color='grey', alpha=0.8, linestyle='--', label='y=x')
        axes[i, j].set_title(f'{name}')
        axes[i, j].set_xlabel('Double mutation')
        axes[i, j].set_ylabel('Multiplication of single mutations')
        axes[i, j].legend()
        axes[i, j].grid(True)
    plt.tight_layout()
    plt.savefig(f'../figures/plots_datasets/1std/all_1std_part{num}.svg')

In [86]:
plt.rcParams.update({'font.size': 14})
subsets = []
start = 0
while start != 60:
    subsets.append(suitable_datasets[start:start+12])
    start += 12
subsets.append(suitable_datasets[start:])
for num in range(len(subsets)):
    subset = subsets[num]
    fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(20, 30))
    for index in range(len(subset)):
        dataset = subset[index]
        name = dataset.split('/')[-1].split('.')[0]
        df = pd.read_csv(dataset)
        indices = {}
        for ind, row in df.iterrows():
            mutations = row['mutant'].split(':')
            if len(mutations) == 2:
                indices[(mutations[0], mutations[1], ind)] = [-1, -1]

        for ind, row in df.iterrows():
            mutations = row['mutant'].split(':')
            if len(mutations) == 1:
                for key, value in indices.items():
                    if key[0] == mutations[0]:
                        indices[key][0] = ind
                    if key[1] == mutations[0]:
                        indices[key][1] = ind

        dic_final = {}
        for key, value in indices.items():
            if value[0] != -1 and value[1] != -1:
                dic_final[key] = value

        #df['DMS_score_log'] = np.log1p(df['DMS_score'])
        scaler = MinMaxScaler()
        df['DMS_score_scaled'] = scaler.fit_transform(df['DMS_score'].values.reshape(-1, 1))

        x = []
        y = []
        selected = []
        for key, value in dic_final.items():
            ind = key[2]
            x.append(df.loc[ind, 'DMS_score_scaled'])
            selected.append(key[0] + ':' + key[1])
            ind_first = value[0]
            ind_second = value[1]
            y.append(df.loc[ind_first, 'DMS_score_scaled'] * df.loc[ind_second, 'DMS_score_scaled'])

        # Calculate distances from the line y = x
        distances = np.abs(np.array(y) - np.array(x)) / np.sqrt(2)
        # Calculate mean and standard deviation of the distances
        mean_distance = np.mean(distances)
        std_distance = np.std(distances)
        # Define a threshold for "distinct" points (e.g., 2 standard deviations from the mean)
        threshold = mean_distance + 2 * std_distance
        # Identify points that are "distinct"
        distinct_points = distances > threshold
        distinct_x = np.array(x)[distinct_points.tolist()]
        distinct_y = np.array(y)[distinct_points.tolist()]
        selected_mutations = [element for element, mask in zip(selected, distinct_points.tolist()) if mask]

        i = index // 3
        j = index % 3
        cmap = plt.get_cmap('Paired')
        axes[i, j].scatter(x, y, color=cmap(1), alpha=0.8, label='near points')
        axes[i, j].scatter(distinct_x, distinct_y, color=cmap(0), alpha=0.8, label='distinct points')
        axes[i, j].plot([0, 1], [0, 1], color='grey', alpha=0.8, linestyle='--', label='y=x')
        axes[i, j].set_title(f'{name}')
        axes[i, j].set_xlabel('Double mutation')
        axes[i, j].set_ylabel('Multiplication of single mutations')
        axes[i, j].legend()
        axes[i, j].grid(True)
    plt.tight_layout()
    plt.savefig(f'../figures/plots_datasets/2std/all_2std_part{num}.svg')
    plt.close()

# Calculating Spearman correlation for "distant" points (all models, all datasets)

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
import os

In [3]:
suitable_datasets = []
cols = ['model']
with open('/home/ank24/epistasis_prediction_review/selected_datasets.txt', 'r') as file:
    for line in file:
        dataset = line.strip()
        suitable_datasets.append(dataset)
        cols.append(dataset + '_Spearman_all')
        cols.append(dataset + '_Spearman_doubles')
        cols.append(dataset + '_Spearman_multi')
        cols.append(dataset + '_Spearman_distinct_1std')
        cols.append(dataset + '_Spearman_distinct_2std')
result_all = pd.DataFrame(columns=cols)

In [9]:
dfs = {}
selected_mutations = {}
directory = '../data/ProteinGym_subs_apr_2024'

for filename in os.listdir(directory): # iterate over datasets
    file_path = os.path.join(directory, filename)
    dataset = filename.split('.')[0]
    if os.path.isfile(file_path) and dataset in suitable_datasets:
        df = pd.read_csv(file_path)
        indices = {}
        for ind, row in df.iterrows():
            mutations = row['mutant'].split(':')
            if len(mutations) == 2: # find 2 mutations
                indices[(mutations[0], mutations[1], ind)] = [-1, -1] # indices[mutation 1, mutation 2, index of the pair], and inside we store indices of individual mutations

        for ind, row in df.iterrows():
            mutations = row['mutant'].split(':')
            if len(mutations) == 1: # match indices of the 2 mutations to a pair
                for key, value in indices.items():
                    if key[0] == mutations[0]:
                        indices[key][0] = ind
                    if key[1] == mutations[0]:
                        indices[key][1] = ind

        dic_final = {}
        for key, value in indices.items():
            if value[0] != -1 and value[1] != -1:
                dic_final[key] = value

        scaler = MinMaxScaler()
        df['DMS_score_scaled'] = scaler.fit_transform(df['DMS_score'].values.reshape(-1, 1)) # minmax scale values

        x = []
        y = []
        selected = []
        for key, value in dic_final.items():
            ind = key[2]
            x.append(df.loc[ind, 'DMS_score_scaled']) # score for a pair
            selected.append(key[0] + ':' + key[1])
            ind_first = value[0]
            ind_second = value[1]
            y.append(df.loc[ind_first, 'DMS_score_scaled'] * df.loc[ind_second, 'DMS_score_scaled']) # multiplied individuals

        # Calculate distances from the line y = x
        distances = np.abs(np.array(y) - np.array(x)) / np.sqrt(2)
        # Calculate mean and standard deviation of the distances
        mean_distance = np.mean(distances)
        std_distance = np.std(distances)
        # Define a threshold for "distinct" points (e.g., 2 standard deviations from the mean)
        threshold = mean_distance + 1 * std_distance
        # Identify points that are "distinct"
        distinct_points = distances > threshold
        selected_muts_1 = [element for element, mask in zip(selected, distinct_points.tolist()) if mask] # for 1 std

        threshold = mean_distance + 2 * std_distance
        distinct_points = distances > threshold
        selected_muts_2 = [element for element, mask in zip(selected, distinct_points.tolist()) if mask] # for 2 std
        
        selected_mutations[dataset] = (selected_muts_1, selected_muts_2)
        dfs[dataset] = df# Iterate over all model prediction folders in the directory

In [14]:
def calculate_spearman(result_all, entry_path, model):
    score_columns = []
    score_indices = {}
    for filename in os.listdir(entry_path): # let's just open 1 file and check if there're several rows with predictions (different num of parameters)
        file_path = os.path.join(entry_path, filename)
        dataset = filename.split('.')[0]
        if dataset in suitable_datasets: 
            df = dfs[dataset]
            df_pred = pd.read_csv(file_path)
            for col in df_pred.columns:
                if ('mut' not in col) and ('seq' not in col) and ('DMS_score' not in col):
                    score_columns.append(col)
            break
    for score_column in score_columns:
        new_row = pd.DataFrame(dict(zip(list(result_all.columns), [model + '_' + score_column] + [None]*(result_all.shape[1] - 1))), index=[0])
        result_all = pd.concat([result_all, new_row], ignore_index=True) # SAVE INDICES + SCORE NAMES IN A DICT TO APPEND AFTERWARDS
        score_indices[score_column] = result_all.index[-1]
    for filename in os.listdir(entry_path): # iterate over datasets
        file_path = os.path.join(entry_path, filename)
        dataset = filename.split('.')[0]
        if dataset in suitable_datasets: 
            df = dfs[dataset]
            df_pred = pd.read_csv(file_path)
            if 'mutant' in df_pred.columns:
                if 'DMS_score' in df_pred.columns:
                    merged_df = df.merge(df_pred, left_on=['mutant', 'DMS_score'], right_on=['mutant', 'DMS_score'])
                else:
                    merged_df = df.merge(df_pred, left_on='mutant', right_on='mutant')
            else:
                if 'DMS_score' in df_pred.columns:
                    merged_df = df.merge(df_pred, left_on=['mutated_sequence', 'DMS_score'], right_on=['mutated_sequence', 'DMS_score'])
                else:
                    merged_df = df.merge(df_pred, left_on='mutated_sequence', right_on='mutated_sequence')

            for score_column in score_columns: # calculate for all predicted scores
                spearman_all = spearmanr(merged_df['DMS_score'], merged_df[score_column])[0]

                merged_df['num_mutations'] = merged_df['mutant'].str.split(':').apply(len)

                merged_df_doubles = merged_df[merged_df['num_mutations'] == 2]
                spearman_doubles = spearmanr(merged_df_doubles['DMS_score'], merged_df_doubles[score_column])[0]

                merged_df_multi = merged_df[merged_df['num_mutations'] >= 2]
                spearman_multi = spearmanr(merged_df_multi['DMS_score'], merged_df_multi[score_column])[0]
                
                selected_mutations_1, selected_mutations_2 = selected_mutations[dataset]
                merged_df_selected_1 = merged_df[merged_df['mutant'].isin(selected_mutations_1)]
                spearman_distinct_1 = spearmanr(merged_df_selected_1['DMS_score'], merged_df_selected_1[score_column])[0]
                merged_df_selected_2 = merged_df[merged_df['mutant'].isin(selected_mutations_2)]
                spearman_distinct_2 = spearmanr(merged_df_selected_2['DMS_score'], merged_df_selected_2[score_column])[0]

                result_all.loc[score_indices[score_column], dataset + '_Spearman_all'] = f"{spearman_all:.2f}"
                result_all.loc[score_indices[score_column], dataset + '_Spearman_doubles'] = f"{spearman_doubles:.2f}"
                result_all.loc[score_indices[score_column], dataset + '_Spearman_multi'] = f"{spearman_multi:.2f}"
                result_all.loc[score_indices[score_column], dataset + '_Spearman_distinct_1std'] = f"{spearman_distinct_1:.2f}"
                result_all.loc[score_indices[score_column], dataset + '_Spearman_distinct_2std'] = f"{spearman_distinct_2:.2f}"
    return result_all

In [15]:
directory = '../data/Predictions'
for model in os.listdir(directory): # iterate over models
    entry_path = os.path.join(directory, model)
    if any(os.path.isdir(os.path.join(entry_path, entry)) for entry in os.listdir(entry_path)): # case when there are subfolders inside a model (different parameter number)
        for folder_name in os.listdir(entry_path): # iterate over submodels
            folder_path = os.path.join(entry_path, folder_name)
            result_all = calculate_spearman(result_all, folder_path, model + '_' + folder_name)
    else: # standart case
        result_all = calculate_spearman(result_all, entry_path, model)
result_all.to_csv('../all_Spearman_values.csv')

# Add linreg baseline

In [5]:
result_all = pd.read_csv('../all_Spearman_values.csv', dtype=str)
result_all.drop(columns='Unnamed: 0', inplace=True)

In [11]:
models = dict(zip(['LinearRegression'], [LinearRegression]))
indices = dict(zip(['LinearRegression'], [len(result_all)]))

In [12]:
for dataset in suitable_datasets:
    df = dfs[dataset]
    df['num_mutations'] = df['mutant'].str.split(':').apply(len)
    enc = preprocessing.OneHotEncoder(handle_unknown="ignore")

    max_len = max(len(seq) for seq in df.mutated_sequence)

    sequences = list(df[df['mutant'].apply(lambda x: len(x.split(':')) == 1)].mutated_sequence)
    
    sequences = [list(seq) for seq in sequences]
    padded_sequences = [seq + [''] * (max_len - len(seq)) for seq in sequences]
    padded_array = np.array(padded_sequences)
    enc.fit(padded_array)
    x_train = enc.transform(padded_array).toarray()
    y_train = df[df['mutant'].apply(lambda x: len(x.split(':')) == 1)].DMS_score.values.reshape(-1,1)

    selected_mutations_1, selected_mutations_2 = selected_mutations[dataset]

    x_test_doubles = y_test_doubles = x_test_multi = y_test_multi = x_test_1 = y_test_1 = x_test_2 = y_test_2 = None

    sequences = df[df['num_mutations'] == 2].mutated_sequence
    if len(sequences) != 0:
        sequences = [list(seq) for seq in sequences]
        padded_sequences = [seq + [''] * (max_len - len(seq)) for seq in sequences]
        padded_array = np.array(padded_sequences)
        x_test_doubles = enc.transform(padded_array).toarray()
        y_test_doubles = df[df['num_mutations'] == 2].DMS_score.values.reshape(-1,1)

    sequences = df[df['num_mutations'] >= 2].mutated_sequence
    if len(sequences) != 0:
        sequences = [list(seq) for seq in sequences]
        padded_sequences = [seq + [''] * (max_len - len(seq)) for seq in sequences]
        padded_array = np.array(padded_sequences)
        x_test_multi = enc.transform(padded_array).toarray()
        y_test_multi = df[df['num_mutations'] >= 2].DMS_score.values.reshape(-1,1)

    sequences = df[df['mutant'].isin(selected_mutations_1)].mutated_sequence
    if len(sequences) != 0:
        sequences = [list(seq) for seq in sequences]
        padded_sequences = [seq + [''] * (max_len - len(seq)) for seq in sequences]
        padded_array = np.array(padded_sequences)
        x_test_1 = enc.transform(padded_array).toarray()
        y_test_1 = df[df['mutant'].isin(selected_mutations_1)].DMS_score.values.reshape(-1,1)

    sequences = df[df['mutant'].isin(selected_mutations_2)].mutated_sequence
    if len(sequences) != 0:
        sequences = [list(seq) for seq in sequences]
        padded_sequences = [seq + [''] * (max_len - len(seq)) for seq in sequences]
        padded_array = np.array(padded_sequences)
        x_test_2 = enc.transform(padded_array).toarray()
        y_test_2 = df[df['mutant'].isin(selected_mutations_2)].DMS_score.values.reshape(-1,1)

    for model in models:
        result_all.loc[indices[model], 'model'] = model

        model_dataset = models[model]()
        model_dataset.fit(x_train, y_train)

        sequences = list(df.mutated_sequence)
        sequences = [list(seq) for seq in sequences]
        padded_sequences = [seq + [''] * (max_len - len(seq)) for seq in sequences]
        padded_array = np.array(padded_sequences)
        x_test = enc.transform(padded_array).toarray()
        y_test = df.DMS_score.values.reshape(-1,1)
        y_pred = model_dataset.predict(x_test)
        spearman = spearmanr(y_test, y_pred)[0]
        result_all.loc[indices[model], dataset + '_Spearman_all'] = spearman

        if x_test_doubles is not None:
            y_pred_doubles = model_dataset.predict(x_test_doubles)
            spearman_doubles = spearmanr(y_test_doubles, y_pred_doubles)[0]
            result_all.loc[indices[model], dataset + '_Spearman_doubles'] = f"{spearman_doubles:.2f}"
        else:
            result_all.loc[indices[model], dataset + '_Spearman_doubles'] = 0

        if x_test_multi is not None:
            y_pred = model_dataset.predict(x_test_multi)
            spearman = spearmanr(y_test_multi, y_pred)[0]
            result_all.loc[indices[model], dataset + '_Spearman_multi'] = f"{spearman:.2f}"
        else:
            result_all.loc[indices[model], dataset + '_Spearman_multi'] = 0

        if x_test_1 is not None:
            y_pred_1 = model_dataset.predict(x_test_1)
            spearman_distinct_1 = spearmanr(y_test_1, y_pred_1)[0]
            result_all.loc[indices[model], dataset + '_Spearman_distinct_1std'] = f"{spearman_distinct_1:.2f}"
        else:
            result_all.loc[indices[model], dataset + '_Spearman_distinct_1std'] = 0

        if x_test_2 is not None:
            y_pred_2 = model_dataset.predict(x_test_2)
            spearman_distinct_2 = spearmanr(y_test_2, y_pred_2)[0]
            result_all.loc[indices[model], dataset + '_Spearman_distinct_2std'] = f"{spearman_distinct_2:.2f}"
        else:
            result_all.loc[indices[model], dataset + '_Spearman_distinct_2std'] = 0

In [16]:
result_all.to_csv('../all_Spearman_values.csv')

# Plots of Spearman values

## All points

In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
import seaborn as sns
import os

In [18]:
suitable_datasets = []
with open('../selected_datasets.txt', 'r') as file:
    for line in file:
        dataset = line.strip()
        suitable_datasets.append(dataset)

In [23]:
df = pd.read_csv('../all_Spearman_values.csv', index_col=1)
df.drop(columns='Unnamed: 0', inplace=True)
for dataset in suitable_datasets:
    all = np.abs(df[dataset + '_Spearman_all'])
    std1 = np.abs(df[dataset + '_Spearman_distinct_1std'])
    std2 = np.abs(df[dataset + '_Spearman_distinct_2std'])
    cmap = plt.get_cmap('Paired')
    #plt.figure(figsize=[34,38])
    categories = df.index  # Coordinates on the x-axis
    x = np.arange(len(categories))  # Positions for the groups
    width = 0.2  # Width of the bars
    fig, ax = plt.subplots(figsize=(28, 8))
    # Plot each set of bars with a specific x offset
    ax.bar(x - width, all, width, color='midnightblue', label='All points')
    ax.bar(x, std1, width, color=cmap(1), label='Distant points (1 std)')
    ax.bar(x + width, std2, width, color=cmap(0), label='Distant points (2 std)')
    # Add labels, title, and legend
    ax.set_xlabel('Model', fontsize=14)
    ax.set_ylabel('Spearman correlation', fontsize=14)
    ax.set_title(f'Spearman correlations for {dataset}', fontsize=16)
    ax.set_xticks(x)
    ax.set_xticklabels(categories)
    ax.grid(True, axis='y')
    plt.xticks(rotation=90)
    plt.legend(prop={'size': 14})
    plt.savefig(f'../figures/plots_models/{dataset}.svg', bbox_inches='tight')
    plt.close()

## Double/multi

In [27]:
df = pd.read_csv('../all_Spearman_values.csv', index_col=1)
df.drop(columns='Unnamed: 0', inplace=True)
for dataset in suitable_datasets:
    doubles = np.abs(df[dataset + '_Spearman_doubles'])
    multi = np.abs(df[dataset + '_Spearman_multi'])
    std1 = np.abs(df[dataset + '_Spearman_distinct_1std'])
    std2 = np.abs(df[dataset + '_Spearman_distinct_2std'])
    cmap = plt.get_cmap('Paired')
    #plt.figure(figsize=[34,38])
    categories = df.index  # Coordinates on the x-axis
    x = np.arange(len(categories))  # Positions for the groups
    width = 0.2  # Width of the bars
    fig, ax = plt.subplots(figsize=(28, 8))
    # Plot each set of bars with a specific x offset
    ax.bar(x - 1.5*width, doubles, width, color=cmap(6), label='Double mutations')
    ax.bar(x - width/2, multi, width, color=cmap(7), label='Multi (2+) mutations')
    ax.bar(x + width/2, std1, width, color=cmap(1), label='Distant points (1 std)')
    ax.bar(x + 1.5*width, std2, width, color=cmap(0), label='Distant points (2 std)')
    # Add labels, title, and legend
    ax.set_xlabel('Model', fontsize=14)
    ax.set_ylabel('Spearman correlation', fontsize=14)
    ax.set_title(f'Spearman correlations for {dataset}', fontsize=16)
    ax.set_xticks(x)
    ax.set_xticklabels(categories)
    ax.grid(True, axis='y')
    plt.xticks(rotation=90)
    plt.legend(prop={'size': 14})
    plt.savefig(f'../figures/plots_models/{dataset}_multi.svg', bbox_inches='tight')
    plt.close()

# Violin plots

In [37]:
print(np.isinf(df_slice).sum())

A4_HUMAN_Seuma_2022_Spearman_distinct_2std               0
AMFR_HUMAN_Tsuboyama_2023_4G3O_Spearman_distinct_2std    0
BBC1_YEAST_Tsuboyama_2023_1TG0_Spearman_distinct_2std    0
BCHB_CHLTE_Tsuboyama_2023_2KRU_Spearman_distinct_2std    0
CAPSD_AAV2S_Sinai_2021_Spearman_distinct_2std            0
                                                        ..
UBR5_HUMAN_Tsuboyama_2023_1I2T_Spearman_distinct_2std    0
VILI_CHICK_Tsuboyama_2023_1YU5_Spearman_distinct_2std    0
YAIA_ECOLI_Tsuboyama_2023_2KVT_Spearman_distinct_2std    0
YAP1_HUMAN_Araya_2012_Spearman_distinct_2std             0
YNZC_BACSU_Tsuboyama_2023_2JVD_Spearman_distinct_2std    0
Length: 69, dtype: int64


In [54]:
import seaborn as sns

def rename_columns(col, num):
    parts = col.split('_')  # Split by '_'
    return '_'.join(parts[:len(parts) - num])

In [60]:
plots = ['_Spearman_all', '_Spearman_doubles', '_Spearman_multi', '_Spearman_distinct_1std', '_Spearman_distinct_2std']
cnt = 0
for plot in plots:
    df_slice = np.abs(df.filter(like=plot))
    if cnt < 3:
        df_slice.columns = [rename_columns(col, 2) for col in df_slice.columns]
    else:
        df_slice.columns = [rename_columns(col, 3) for col in df_slice.columns]
    cnt += 1
    df_slice.fillna(0, inplace=True)
    plt.figure(figsize=[28,8])
    sns.violinplot(df_slice)
    plt.xlabel('Datasets', fontsize=14)
    plt.ylabel('Spearman correlation', fontsize=14)
    plt.title(f'Spearman correlation of all methods ({plot.split('_')[-1]} points)', fontsize=16)
    plt.xticks(rotation=90)
    plt.grid(True, axis='y')
    plt.savefig(f'../figures/violin_plots/{plot[1:]}.svg', bbox_inches='tight')
    plt.close()